In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

import sys
sys.path.append("..")
from utils import utils, model_utils, find_all_colnames
import fine_tuning, training_summary

In [2]:
checkpoints_path = "E:/TFM_CCIL/checkpoints_colab/"
task = input("Task: ")
experiment = input("Experiment: ")
save_to = input("Save to: ")

table = training_summary.make_train_summary_table(checkpoints_path, task, experiment, save_to)

Task: sentiment
Experiment: acl
Save to: _
Updated utils/all_colnames.tsv


In [3]:
output_path = input("Output path: ")
training_summary.make_train_summary_plots(task, output_path=output_path)